# Введение в NLP, часть 2


## NER c BERT (25 баллов)

В данном задании мы будем обучать BERT на задаче Named Entity Recognition.

### Подготовка данных (5 баллов)

Подумать о:
1. Как subword токенизация повлияет на BIO раззметку?
2. Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на «слова» текстами

In [1]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)

In [2]:
conll2003 = load_dataset("eriktks/conll2003", trust_remote_code=True)
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* ner_tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем


Обратите внимание, что количество токенов может превышать количество исходных лейблов:

In [4]:
bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()

['[CLS]',
 'Ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

Значение тэга в `ner_tags` отображается в метку NER:

In [5]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)


In [6]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])
tags_str = []
features = conll2003["train"].features["ner_tags"].feature

for tag in example["ner_tags"]:
    tags_str.append(features.int2str(tag))
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке IOB. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как 

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `ner_tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `ner_tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["ner_tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [7]:
from itertools import groupby

def preprocess_ner_dataset(example):
    tokens = example["tokens"]
    ner_tags = example["ner_tags"]
    
    tokenized = bert_tokenizer(tokens, is_split_into_words=True, return_offsets_mapping=True)
    
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]
    token_type_ids = tokenized["token_type_ids"]
    
    labels = []
    word_ids = tokenized.word_ids()
    
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)  # спецтокены
        else:
            if word_idx != previous_word_idx:
                labels.append(ner_tags[word_idx])  # оригинальный тег слова
            else:
                # если первый токен был 'B-*', то делаем 'I-*'
                prev_tag = ner_tags[word_idx]
                if features.int2str(prev_tag).startswith("B-"):
                    labels.append(features.str2int("I-" + features.int2str(prev_tag)[2:]))
                else:
                    labels.append(prev_tag)  # иначе просто повторяем тег
        previous_word_idx = word_idx

    # Проверяем размерности
    assert len(labels) == len(input_ids), f"Mismatch: {len(labels)} labels and {len(input_ids)} tokens"

    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [8]:
assert preprocess_ner_dataset(conll2003["train"][0]) == {
    'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
    'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
}

### Тесты

In [9]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [10]:
from tqdm import tqdm
for idx, example in tqdm(enumerate(conll2003["train"])):
    input_ids_real = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    input_ids_ours = preprocess_ner_dataset(example)["input_ids"]
    assert input_ids_real == input_ids_ours, f"Ошибка токенизации на примере {idx}"
    if idx >= 10:
        break
print("Токенизация верна!")

10it [00:00, 1432.09it/s]

Токенизация верна!


In [11]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [12]:
example = conll2003["train"][200]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

Применим нашу функцию к всему датасету

In [13]:
preprocessed_ner_dataset = conll2003.map(preprocess_ner_dataset, num_proc=1)
preprocessed_ner_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [14]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)
type(data_collator)

transformers.data.data_collator.DataCollatorForTokenClassification

### Подготовка модели (5 баллов)

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForTokenClassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется). Ему нужно подать num_labels - число классов (число возможных меток для токенов), id2label - словарь индекс метки -> текст метки , label2id - словарь текст метки -> индекс метки. Из-за непростого наследования эти аргументы тяжело найти, прочитать про них можно [тут](https://huggingface.co/docs/transformers/en/main_classes/configuration#transformers.PretrainedConfig)
2. Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860).

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["ner_tags"].feature.num_classes` классов.


In [16]:
from transformers import AutoModelForTokenClassification, AutoModel

label_names = conll2003["train"].features["ner_tags"].feature.names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

bert_ner = AutoModelForTokenClassification.from_pretrained(
    BASE_NER_MODEL, 
    num_labels=conll2003["train"].features["ner_tags"].feature.num_classes,
    id2label=id2label, 
    label2id=label2id,
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Подготовим Метрику (5 баллов)

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет. В этом нам поможет argmax.
2. Убрать паддинги - это те позиции, где метки (labels) равняются -100
3. Перевести labels из чисел в текстовые метки с помощью `label_list`
4. Посчитать метрики с помощью `metrics_calculator`
5. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики. Брать можно только `overall_*` метрики для удобства

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [17]:
label_list = conll2003["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [18]:
import numpy as np
import evaluate


metrics_calculator = evaluate.load("seqeval")
label_list = conll2003["train"].features["ner_tags"].feature.names

y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

print(metrics_calculator.compute(references=y_true, predictions=y_pred))


{'MISC': {'precision': np.float64(0.0), 'recall': np.float64(0.0), 'f1': np.float64(0.0), 'number': np.int64(1)}, 'PER': {'precision': np.float64(1.0), 'recall': np.float64(1.0), 'f1': np.float64(1.0), 'number': np.int64(1)}, 'overall_precision': np.float64(0.5), 'overall_recall': np.float64(0.5), 'overall_f1': np.float64(0.5), 'overall_accuracy': 0.8}


In [20]:
def calculate_metrics(eval_predictions):
    logits, labels = eval_predictions

    predictions = np.argmax(logits, axis=-1)

    true_labels = []
    pred_labels = []
    
    for label_seq, pred_seq in zip(labels, predictions):
        true_seq = []
        pred_seq_filtered = []
        for label, pred in zip(label_seq, pred_seq):
            if label != -100:
                true_seq.append(label_list[label])
                pred_seq_filtered.append(label_list[pred])
        true_labels.append(true_seq)
        pred_labels.append(pred_seq_filtered)

    results = metrics_calculator.compute(references=true_labels, predictions=pred_labels)

    return {key: value for key, value in results.items() if key.startswith("overall")}


import numpy as np

logits = np.array([
    [  # Batch 1
        [7, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
        [7, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
        [0, 0, 0, 0, 0, 0, 0, 7, 0],  # 'B-MISC'
        [0, 0, 0, 0, 0, 0, 0, 0, 7],  # 'I-MISC'
        [0, 0, 0, 0, 0, 0, 0, 0, 7],  # 'I-MISC'
        [0, 0, 0, 0, 0, 0, 0, 0, 7],  # 'I-MISC'
        [7, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
    ],
    [  # Batch 2
        [0, 12, 0, 0, 0, 0, 0, 0, 0],  # 'B-PER'
        [0, 0, 10, 0, 0, 0, 0, 0, 0],  # 'I-PER'
        [12, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
        [0, 12, 0, 0, 0, 0, 0, 0, 0],  # 'B-PER' - но не оно по паддингам, поэтому не должно считаться в метрике!
        [0, 0, 12, 0, 0, 0, 0, 0, 0],  # 'I-PER'
        [0, 0, 12, 0, 0, 0, 0, 0, 0],  # 'I-PER'
        [0, 0, 12, 0, 0, 0, 0, 0, 0],  # 'I-PER'
    ]
])

labels = np.array([
    [0, 0, 0, 7, 8, 8, 0],  # 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'
    [1, 2, 0, -100, -100, -100, -100]  # 'B-PER', 'I-PER', 'O' (with padding -100s)
])

logits_res = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"], ["B-PER", "I-PER", "O"]]
labels_res = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"], ["B-PER", "I-PER", "O"]]

metrics_on_tensors = calculate_metrics((logits, labels))
print(metrics_on_tensors)
metrics_on_strs = metrics_calculator.compute(references=logits_res, predictions=labels_res)

for k in metrics_on_tensors:
    if "overall" in k:
        assert metrics_on_tensors[k] == metrics_on_strs[k], k


{'overall_precision': np.float64(0.5), 'overall_recall': np.float64(0.5), 'overall_f1': np.float64(0.5), 'overall_accuracy': 0.8}


### Обучение (5 баллов)

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html) класс из `transformers`
2. Написать свой training loop

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [30]:
import torch
print(torch.cuda.is_available())

False


In [32]:
from transformers import Trainer, TrainingArguments

# Определяем параметры обучения
args = TrainingArguments(
    output_dir = "./results",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    metric_for_best_model = "overall_f1",
    greater_is_better = True,
    learning_rate = 2e-5,
    num_train_epochs = 3,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    save_total_limit = 2,
    load_best_model_at_end = True,
)

trainer = Trainer(
    model = bert_ner,
    args = args,
    train_dataset = preprocessed_ner_dataset["train"],
    eval_dataset = preprocessed_ner_dataset["validation"],
    compute_metrics = calculate_metrics,
    data_collator = data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.073700,0.074339,0.896260,0.927634,0.911677,0.980441
2,0.033100,0.071227,0.934377,0.944127,0.939227,0.984738
3,0.019900,0.063235,0.931120,0.948670,0.939813,0.986078


TrainOutput(global_step=5268, training_loss=0.05200552922265495, metrics={'train_runtime': 6539.6404, 'train_samples_per_second': 6.441, 'train_steps_per_second': 0.806, 'total_flos': 920771584279074.0, 'train_loss': 0.05200552922265495, 'epoch': 3.0})

In [33]:
*_, metrics = trainer.predict(preprocessed_ner_dataset['test'])
metrics

{'test_loss': 0.19507157802581787,
 'test_overall_precision': 0.8865820178448868,
 'test_overall_recall': 0.9148371104815864,
 'test_overall_f1': 0.9004879749041479,
 'test_overall_accuracy': 0.9715730921353272,
 'test_runtime': 94.5455,
 'test_samples_per_second': 36.522,
 'test_steps_per_second': 4.569}

### Обработка результатов Результатов (5 баллов)

Подумать о:
1. Во время подготовки данных мы приобразовали BIO разметку. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

Можете сравнить результат с [лидербордом](https://paperswithcode.com/sota/token-classification-on-conll2003).


Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}
```

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

1. Вначале нужно токенизировать текст. На вход принимается любой текст без **предварительной токенизации**!
2. Текст нужно прогнать через модель, после чего получатся логиты классов, с помощью которых можно получить индексы меток
3. После этого стоит декодировать сущность и записать ее координаты. Сущность начинается или с B-метки или с метки I- при условии, что сменился класс. Т.е. метки B-PER, I-PER, I-MISC должны декодироваться 2 сущности PER и MISC!


In [34]:
bert_tokenizer("Hello world", return_offsets_mapping=True, return_tensors="pt")

{'input_ids': tensor([[ 101, 8667, 1362,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]]), 'offset_mapping': tensor([[[ 0,  0],
         [ 0,  5],
         [ 6, 11],
         [ 0,  0]]])}

In [48]:
from typing import List, Tuple
text = "Иван Иванов caterpillar Apple Стив Джобс"
labels = ["I-PER", "I-PER", "I-MISC", "B-MISC", "B-PER", "B-PER"]
offsets = [(0, 4), (5, 11), (12, 23), (24, 29), (30, 34), (35, 40)]

def decode_entities(text: str, labels: List[str], offsets: List[Tuple[int, int]]):
    entities = []
    current_entity = None
    current_class = None
    
    for i, (label, (start, end)) in enumerate(zip(labels, offsets)):
        if label.startswith("B-"):
            if current_entity:
                entities.append(current_entity)
            current_class = label[2:]
            current_entity = {"class": current_class, "text": text[start:end], "start": start, "end": end}
        elif label.startswith("I-"):
            entity_class = label[2:]
            if current_entity and current_class == entity_class:
                current_entity["text"] += " " + text[start:end]
                current_entity["end"] = end
            else:
                if current_entity:
                    entities.append(current_entity)
                current_class = entity_class
                current_entity = {"class": current_class, "text": text[start:end], "start": start, "end": end}
    
    if current_entity:
        entities.append(current_entity)
    
    return {"text": text, "entities": entities}

assert decode_entities(text, labels, offsets) == {
    'text': 'Иван Иванов caterpillar Apple Стив Джобс',
    'entities': [
        {'class': 'PER', 'text': 'Иван Иванов', 'start': 0, 'end': 11},
        {'class': 'MISC', 'text': 'caterpillar', 'start': 12, 'end': 23},
        {'class': 'MISC', 'text': 'Apple', 'start': 24, 'end': 29},
        {'class': 'PER', 'text': 'Стив', 'start': 30, 'end': 34},
        {'class': 'PER', 'text': 'Джобс', 'start': 35, 'end': 40}
    ]
}

В функции do_ner нужно токенизировать текст, получить выходы модели на тексте и подать все в decode_entities

In [51]:
@torch.no_grad()
def do_ner(text: str):
    bert_ner.eval()
    
    tokens = bert_tokenizer(text, return_offsets_mapping=True, return_tensors="pt", is_split_into_words=False)
    input_ids = tokens["input_ids"]
    offsets = tokens["offset_mapping"].tolist()[0]
    
    outputs = bert_ner(input_ids).logits
    predicted_labels = torch.argmax(outputs, dim=-1)[0].tolist()
    
    label_names = [bert_ner.config.id2label[idx] for idx in predicted_labels]
    
    valid_offsets = [(start, end) for (start, end) in offsets if start != end]
    
    return decode_entities(text, label_names, valid_offsets)

# Тестовый пример
print(do_ner("Ivan Petrov is going to start working tomorrow"))

{'text': 'Ivan Petrov is going to start working tomorrow', 'entities': [{'class': 'PER', 'text': 'Ivan Pet rov', 'start': 0, 'end': 11}]}


Почистим память перед второй частью.

In [54]:
import torch

del bert_ner
del trainer
torch.cuda.empty_cache()

NameError: name 'bert_ner' is not defined

## Классификация с T5 (25 баллов)

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать тексты для обозначения классов
2. Обучить t5-small на генерацию выбранных названия классов
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных (6 баллов)

Подумать о:
1) Какой префикс выбрать для новой задачи?
2) Должен ли префикс быть понятным?
3) Как выбрать метку для класса? 
4) Что будет, если метки класса целиком нет в словаре?
5) Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным.}

In [55]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

toxic-chat_annotation_train.csv:   0%|          | 0.00/8.20M [00:00<?, ?B/s]

toxic-chat_annotation_test.csv:   0%|          | 0.00/8.09M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Место для изучения датасета:

In [56]:
toxic_chat_dataset["train"][0]

{'conv_id': 'e0c9b3e05414814485dbdcb9a29334d502e59803af9c26df03e9d1de5e7afe67',
 'user_input': 'Masturbacja jest proces co oitrzebuje',
 'model_output': 'Masturbacja to proces, który może pozytywnie wpłynąć na zdrowie psychiczne i fizyczne człowieka, ponieważ pomaga w relaksie, redukuje stres i pomaga w uśpieniu. Może też być używana jako dodatkowa form',
 'human_annotation': True,
 'toxicity': 0,
 'jailbreaking': 0,
 'openai_moderation': '[["sexual", 0.4609803557395935], ["sexual/minors", 0.0012527990620583296], ["harassment", 0.0001862536446424201], ["hate", 0.00015521160094067454], ["violence", 6.580814078915864e-05], ["self-harm", 3.212967567378655e-05], ["violence/graphic", 1.5190824342425913e-05], ["self-harm/instructions", 1.0009921425080393e-05], ["hate/threatening", 4.4459093260229565e-06], ["self-harm/intent", 3.378846486157272e-06], ["harassment/threatening", 1.7095695739044459e-06]]'}

Нас будут интересовать колонки `"user_input"` и `"toxicity"`. Убираем ненужные колонки из датасета:

In [57]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов есть на картинке выше - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1) Не забыть добавить префикс перед токенизацией входного текста
2) Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [68]:
PREFIX = "Classify toxicity: "
MAX_LENGTH = 512

id2label = {
    0: "Этот текст безопасен",
    1: "Этот текст токсичен"
}

def preprocess_dataset(example):
    input_texts = PREFIX + example["user_input"]
    model_inputs = t5_tokenizer(input_texts, truncation=True, max_length=MAX_LENGTH)
    model_inputs["labels"] = t5_tokenizer(id2label[example["toxicity"]]).input_ids
    return model_inputs

toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset)

toxic_chat_dataset

Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['user_input', 'toxicity', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5082
    })
    test: Dataset({
        features: ['user_input', 'toxicity', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5083
    })
})

In [69]:
toxic_chat_dataset['train'][0]

{'user_input': 'Masturbacja jest proces co oitrzebuje',
 'toxicity': 0,
 'input_ids': [4501,
  4921,
  3,
  27147,
  10,
  6664,
  2905,
  9305,
  1191,
  528,
  7,
  17,
  6345,
  576,
  3,
  32,
  155,
  52,
  776,
  3007,
  1924,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [3,
  2,
  9592,
  6725,
  3,
  14982,
  6652,
  10458,
  3,
  2,
  1757,
  2,
  2044,
  2,
  2533,
  5345,
  22123,
  1]}

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```
Значения в `'labels'` в вашем случае могут отличаться, это зависит от выбранного вами текстового представления в `id2label` словаре.


Инициализируем соответствующий задаче `DataCollator`.

In [73]:
from transformers import DataCollatorForSeq2Seq


data_collator = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=seq2seq_model)

### Определим метрику (2 балла)

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию. Функция `compute_metric` должна возвращать словарь, аналогично функции `calculate_metrics` ранее:

```json
{
    "accuracy": значение точности,
    ...
}
```

Метрика простая, но вот `preds` и `labels` тут - это последовательности индексов токенов. Нужно это учесть.

In [79]:
import torch

def compute_metric(eval_predictions):
    preds, labels = eval_predictions
    
    # Преобразуем в тензоры PyTorch (если передан np.ndarray)
    preds = torch.tensor(preds) if not isinstance(preds, torch.Tensor) else preds
    labels = torch.tensor(labels) if not isinstance(labels, torch.Tensor) else labels

    preds = preds[:, 1]  # Убираем токен начала последовательности
    labels = labels[:, 0]  # Берем первый токен из метки

    accuracy = (preds == labels).float().mean().item()
    return {"accuracy": accuracy}

def check_compute_metric():
    # два предсказания, где токен 150 обозначает токсичный лейбл, токен 120 - нетоксичный лейбл
    preds = torch.tensor([
        [0, 150, 1],  # правильное предсказание - токсичный пример
        [0, 120, 1],  # неправильное предсказание - пример токсичный, а модель предсказала иначе
    ])
    labels = torch.tensor([
        [150, 1],
        [150, 1],
    ])
    assert torch.isclose(
        torch.tensor(compute_metric((preds.numpy(), labels.numpy()))["accuracy"], dtype=torch.double),
        torch.tensor(0.5, dtype=torch.double),
    )

check_compute_metric()


### Определить Модель (2 балла)

Инициализируйте модель из базового чекпоинта

In [75]:
from transformers import AutoModelForSeq2SeqLM


seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL)

### Обучение (10 баллов)

Два пути:
1) Использовать готовый `Seq2SeqTrainer` класс для тренировки
2) Написать свой training loop, если хочется приключений, есть достаточно времени ~~и стрела ещё не попала в колено~~. Дополнительных баллов за это не будет

> Hint! Обратите внимание на функцию `seq2seq_model._shift_right` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1) Что подаётся на вход в энкодер?
2) Что подаётся на вход в декодер?
3) Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4) Как используется выход энкодера в декодере?

In [80]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_tox",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=seq2seq_model,
    args=training_args,
    train_dataset=toxic_chat_dataset["train"],
    eval_dataset=toxic_chat_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metric,
)

trainer.train()

c:\Users\Professional\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.011900,0.011192,1.000000
2,0.011600,0.009449,1.000000
3,0.010800,0.008688,1.000000


TrainOutput(global_step=1908, training_loss=0.011153626242023843, metrics={'train_runtime': 5379.7417, 'train_samples_per_second': 2.834, 'train_steps_per_second': 0.355, 'total_flos': 760407866671104.0, 'train_loss': 0.011153626242023843, 'epoch': 3.0})

### Сравнение Результатов (5 баллов)

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`. Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета? 

Подумать о:
1) В чём преимущество такого подхода к классификации?
2) В чём недостатки такого подхода к классификации?
3) Как ещё можно решать классификационные задачи с помощью t5?

In [81]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

negative


Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.

In [83]:
def is_toxic(
    text: str,
    labels2bool,
    model=trainer.model,
    tokenizer=t5_tokenizer,
    prefix=PREFIX,
) -> bool:
    model.eval()
    input_text = prefix + text
    inputs = tokenizer(input_text, return_tensors="pt")
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs)
    
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return labels2bool.get(decoded_output, False)

assert not is_toxic(
    text="This is just a text",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prefix=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)




False